# Class handout Questions and Discussion

The following is plots that discuss questions in the class handout for meeting 16.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

github_location = "https://raw.githubusercontent.com/blinkletter/4410PythonNotebooks/main/Class_16/data/"
github_location_styles = "https://raw.githubusercontent.com/blinkletter/LFER-QSAR/main/styles/"
github_location_LFER_tables = "https://raw.githubusercontent.com/blinkletter/LFER-QSAR/main/data/"

#################################################################
### a function to fill in sigma for empty spaces in s+ and s- ###
#################################################################

def fill_sigma(df):     
    for z in df.index:
        if np.isnan(df.loc[z,"s_plus"]):
            df.loc[z,"s_plus"] = df.loc[z,"sigma"]
        if np.isnan(df["s_minus"][z]):
            df.loc[z,"s_minus"] = df.loc[z,"sigma"]
    return(df)



################################################################################
### Read data set. The fields are separated by commas; comments are enabled  ###
################################################################################

LFER_file = "LFER_HanschLeoTaft.csv"
#LFER_file = "Taft_Es_Williams.csv"

data_set = pd.read_csv(github_location_LFER_tables + LFER_file,
                 delimiter = ",", 
                 skipinitialspace=True, 
                 index_col="Substituent", 
                 comment = "#") 

########################################################
### Fill across sigma values and select substituents ###
########################################################

data_set=fill_sigma(data_set)

### Remove unneeded columns
data_set.drop(labels = ["TABLE V", "TABLE I"], axis = 1, inplace = True)

data_set.sort_values(by=['sigma'], inplace=True)

# print(data_set)

################################################################################
### Read table 1 data. The fields are separated by commas; comments are enabled  ###
################################################################################

data_file = "Table_1_data.csv"

table_data_df = pd.read_csv(github_location + data_file,
                 delimiter = ",", 
                 skipinitialspace=True, 
                 index_col="Substituent", 
                 comment = "#") 


             sigma  s_plus  s_minus
Substituent                        
p-N(CH3)2    -0.83   -1.70    -0.12
p-NH2        -0.66   -1.30    -0.15
p-OH         -0.37   -0.92    -0.37
p-OCH3       -0.27   -0.78    -0.26
p-OEt        -0.24   -0.81    -0.28
p-CMe3       -0.20    0.26    -0.13
p-CH3        -0.17   -0.31    -0.17
m-NH2        -0.16   -0.16    -0.16
m-N(CH3)2    -0.15   -0.15    -0.15
p-Et         -0.15   -0.30    -0.19
m-CMe3       -0.10   -0.10    -0.10
m-CH3        -0.07   -0.07    -0.07
m-Et         -0.07   -0.07    -0.07
p-OC6H5      -0.03   -0.50    -0.10
p-C6H5       -0.01    0.02    -0.18
H             0.00    0.00     0.00
p-CCCH3       0.03   -0.26    -0.13
p-CHCH2       0.04   -0.16     0.04
p-F           0.06   -0.07    -0.03
m-CHCH2       0.06    0.06     0.06
m-C6H5        0.06    0.06     0.06
p-N3          0.08    0.08     0.11
m-OEt         0.10    0.10     0.10
m-OCH3        0.12    0.12     0.12
m-OH          0.12    0.12     0.12
p-I           0.18    0.14  